**[Pandas Course Home Page](https://www.kaggle.com/learn/pandas)**

---


# Intro

This is the workbook component to the "Summary functions and maps" section of the Advanced Pandas tutorial. For the reference section, [**click here**](https://www.kaggle.com/residentmario/summary-functions-and-maps-reference).

In the last section we learned how to select relevant data out of our `pandas` `DataFrame` and `Series` objects. Plucking the right data out of our data representation is critical to getting work done, as we demonstrated in the visualization exercises attached to the workbook.

However, the data does not always come out of memory in the format we want it in right out of the bat. Sometimes we have to do some more work ourselves to reformat it for the task at hand.

The remainder of this tutorial will cover different operations we can apply to our data to get the input "just right". We'll start off in this section by looking at the most commonly looked built-in reshaping operations. Along the way we'll cover data `dtypes`, a concept essential to working with `pandas` effectively.

In [1]:
import pandas as pd
pd.set_option("display.max_rows", 5)
reviews = pd.read_csv("../input/wine-reviews/winemag-data-130k-v2.csv", index_col=0)

from learntools.core import binder; binder.bind(globals())
from learntools.pandas.summary_functions_and_maps import *
print("Setup complete.")

reviews.head()

Setup complete.


,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


## Exercises

## 1.

What is the median of the `points` column in the `reviews` DataFrame?

In [2]:
median_points = reviews.points.median()
print(median_points)

q1.check()

88.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [3]:
#q1.hint()
#q1.solution()

## 2. 
What countries are represented in the dataset? (Your answer should not include any duplicates.)

In [4]:
countries = reviews.country.unique()
print(countries)

q2.check()

['Italy' 'Portugal' 'US' 'Spain' 'France' 'Germany' 'Argentina' 'Chile'
 'Australia' 'Austria' 'South Africa' 'New Zealand' 'Israel' 'Hungary'
 'Greece' 'Romania' 'Mexico' 'Canada' nan 'Turkey' 'Czech Republic'
 'Slovenia' 'Luxembourg' 'Croatia' 'Georgia' 'Uruguay' 'England' 'Lebanon'
 'Serbia' 'Brazil' 'Moldova' 'Morocco' 'Peru' 'India' 'Bulgaria' 'Cyprus'
 'Armenia' 'Switzerland' 'Bosnia and Herzegovina' 'Ukraine' 'Slovakia'
 'Macedonia' 'China' 'Egypt']


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [5]:
#q2.hint()
#q2.solution()

## 3.
How often does each country appear in the dataset? Create a Series `reviews_per_country` mapping countries to the count of reviews of wines from that country.

In [6]:
reviews_per_country = reviews.country.value_counts()
print(reviews_per_country)

q3.check()

US          54504
France      22093
            ...  
Egypt           1
Slovakia        1
Name: country, Length: 43, dtype: int64


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [7]:
#q3.hint()
#q3.solution()

## 4.
Create variable `centered_price` containing a version of the `price` column with the mean price subtracted.

(Note: this 'centering' transformation is a common preprocessing step before applying various machine learning algorithms.) 

In [8]:
centered_price = reviews.price - reviews.price.mean()
print(centered_price)

q4.check()

0               NaN
1        -20.363389
            ...    
129969    -3.363389
129970   -14.363389
Name: price, Length: 129971, dtype: float64


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [9]:
#q4.hint()
#q4.solution()

## 5.
I'm an economical wine buyer. Which wine is the "best bargain"? Create a variable `bargain_wine` with the title of the wine with the highest points-to-price ratio in the dataset.

In [10]:
bargain_idx = (reviews.points / reviews.price).idxmax()
bargain_wine = reviews.loc[bargain_idx, 'title']

q5.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [11]:
q5.hint()
q5.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> The [`idxmax` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.idxmax.html) may be useful here.

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
bargain_idx = (reviews.points / reviews.price).idxmax()
bargain_wine = reviews.loc[bargain_idx, 'title']
```

## 6.
There are only so many words you can use when describing a bottle of wine. Is a wine more likely to be "tropical" or "fruity"? Create a Series `descriptor_counts` counting how many times each of these two words appears in the `description` column in the dataset.

In [12]:
check_tropical = reviews.description.map(lambda d : "tropical" in d)
check_fruity = reviews.description.map(lambda d : "fruity" in d)
n_tropical = check_tropical.sum()
n_fruity = check_fruity.sum()
descriptor_counts = pd.Series([n_tropical, n_fruity], index=['tropical', 'fruity'])
print(descriptor_counts)

q6.check()

tropical    3607
fruity      9090
dtype: int64


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [13]:
q6.hint()
q6.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Use a map to check each description for the string `tropical`, then count up the number of times this is `True`. Repeat this for `fruity`. Finally, create a `Series` combining the two values.

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
n_trop = reviews.description.map(lambda desc: "tropical" in desc).sum()
n_fruity = reviews.description.map(lambda desc: "fruity" in desc).sum()
descriptor_counts = pd.Series([n_trop, n_fruity], index=['tropical', 'fruity'])
```

## 7.
We'd like to host these wine reviews on our website, but a rating system ranging from 80 to 100 points is too hard to understand - we'd like to translate them into simple star ratings. A score of 95 or higher counts as 3 stars, a score of at least 85 but less than 95 is 2 stars. Any other score is 1 star.

Also, the Canadian Vintners Association bought a lot of ads on the site, so any wines from Canada should automatically get 3 stars, regardless of points.

Create a series `star_ratings` with the number of stars corresponding to each review in the dataset.

In [14]:
def translate_to_star_ratings(row):
    if row.country == 'Canada':
        return 3
    if row.points > 95:
        return 3
    elif row.points >= 85 and row.points < 95:
        return 2
    else:
        return 1

star_ratings = reviews.apply(translate_to_star_ratings, axis='columns')

q7.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [15]:
#q7.hint()
#q7.solution()

# Keep going
Continue to **[grouping and sorting](https://www.kaggle.com/kernels/fork/598715)**.

---
**[Pandas Course Home Page](https://www.kaggle.com/learn/pandas)**

